# Overview of Ray Tasks

## What is a ray task ?

A ray task is a distributed function in the Ray framework. 'Distributed' means Ray executes the function on a different process and possibly on a separate machine from where you initiate it.

## When to Use Ray Tasks?

You should consider using a ray task in these situations:

- Your code is in Python, Java, or C++ and functions correctly.
- Your code runs slowly because it runs sequentially
- You don't want to rewrite your code to enable this speed up
- You would like to speed up its execution by running it asynchronously
- You aim to scale the code to operate on multiple machines with ease.

Let's look at a code example to make this more concrete.

### Example

We have a Python function conveniently named `expensive_computation`, which executes a computation that requires significant resources and time.

* `expensive_computation` performs a naive matrix multiplication and returns the count of elements in the resulting matrix.

In [ ]:
from itertools import product

Matrix = list[list[int]]

def perform_naive_matrix_multiplication(size: int) -> Matrix:
    matrix1 = matrix2 = [[1 for _ in range(size)] for _ in range(size)]

    result = [[0 for _ in range(size)] for _ in range(size)]
    for i, j, k in product(range(size), range(size), range(size)):
        result[i][j] += matrix1[i][k] * matrix2[k][j]

    return result

def expensive_computation(size: int) -> int:
    result = perform_naive_matrix_multiplication(size)
    n_rows, n_cols = len(result), len(result[0])
    num_elements_in_matrix = n_rows * n_cols
    return num_elements_in_matrix

We require running our matrix multiplication for `n_runs` and then summing the results. We can do this sequentially, but it will take a long time.

In [ ]:
n_runs = 10
size = 300
results = [expensive_computation(size) for _ in range(n_runs)]
assert sum(results) == n_runs * size * size

Below is our code execution visualized 

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/overview/sequential_execution_python_func.svg" width="800px">

#### Desired execution

We would like, instead, to execute our python function in parallel and distribute it over as many machines as possible

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/overview/desired_execution_python_func.svg" width="800px">

Let's see how we can get to this desired using Ray tasks!

## How to define a ray task ?

To define a Ray task, you can use the `@ray.remote` decorator in Python. When you decorate a Python function with `@ray.remote`, it converts the function into a Ray task.

Let's revisit our example and convert our `expensive_computation` function into a Ray task.

In [1]:
import ray


@ray.remote  # decorator to convert python function to ray task
def expensive_computation(size):
    result = perform_naive_matrix_multiplication(size)
    n_rows, n_cols = len(result), len(result[0])
    num_elements_in_matrix = n_rows * n_cols
    return num_elements_in_matrix

### How to run a ray task ?

To run a Ray task, you can use the `.remote` method instead of calling the function directly. The `.remote` method takes the same arguments as the original function.

The `.remote` method is a non-blocking method that returns a future object. A future object is a placeholder for the value that will be returned by the Ray task.

In [ ]:
# submit n_run ray tasks to a ray cluster
# and keep a reference to the task futures
futures = [expensive_computation.remote(size) for _ in range(n_runs)]

### How to fetch the ray task results ?

We use the returned `futures` reference to fetch the result of the function by calling `ray.get(futures)` 

`ray.get` will block until all tasks finish executing and will return the result. 

In [ ]:
# wait for all tasks to complete and get the resulting objects
# results are returned in the same order as submitted
results = ray.get(futures)

In [ ]:
# confirm that we got the right result
assert sum(results) == n_runs * size * size

Here is our execution visualized using Ray tasks. Do you spot the resemblance with our desired execution diagram?

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/overview/overview_distributed_ray_task.svg" width="800px">

# Understanding Ray Task Execution

Let's explore in more detail how the `expensive_computation` function executes in parallel.

Key Points to Note:
- **Ray Tasks** are executed within a **Ray Cluster** as part of a **Ray Job**.
   - A **Ray Job** can be thought of as a collection of tasks, objects, and actors that originate from the same runtime environment.
- **Ray Worker Processes** are the processes responsible for executing the tasks.
- In Ray, futures are referred to as `ObjectRef`s, short for **Object References**.
- Results are stored as **Objects** in the Ray framework.
- The function `ray.get(future)` is used to wait for and retrieve the **Object Value** from a given **Object Reference**.

Here is a more detailed view of the parallel execution

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/overview/overview_detailed_execution_ray_task.svg" width="1000px">

We submit the task and inspect the future object reference - we see that it is a ray.ObjectRef with a given id

In [ ]:
future_object_ref = expensive_computation.remote(size=600)
future_object_ref

This will return something like `ObjectRef(359ec6ce30d3ca2dffffffffffffffffffffffff0100000001000000)`

## What are the state-transitions of a task ?

We now request the cluster state to see our task running and transitioning through some of its states

In [ ]:
from ray.util.state import get_task
import time

start_time = time.time()
duration = 30
while time.time() - start_time < duration:
    time.sleep(5)
    task = get_task(id=future_object_ref.task_id().hex())
    print(
        f"task {task.name} is in state={task.state} "
        f"running on worker {task.worker_id[:8]} as part "
        f"of Job ID {task.job_id}"
    )

This will print out something like 

```text
task expensive_computation is in state=RUNNING running on worker a241e616 as part of Job ID 01000000
task expensive_computation is in state=RUNNING running on worker a241e616 as part of Job ID 01000000
task expensive_computation is in state=RUNNING running on worker a241e616 as part of Job ID 01000000
task expensive_computation is in state=FINISHED running on worker a241e616 as part of Job ID 01000000
task expensive_computation is in state=FINISHED running on worker a241e616 as part of Job ID 01000000
task expensive_computation is in state=FINISHED running on worker a241e616 as part of Job ID 01000000
```

Finally, we use `ray.get` to fetch the resulting object value. It should execute immediately given the task is in a finished state

In [ ]:
object_value = ray.get(future_object_ref)
assert object_value == 360000

In general this diagram below visualizes the normal order of state transitions that a task is expected to go through

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/overview/state_transition.svg" width="800px">

Note that given the PENDING and SUBMITTED_TO_WORKER states are very fast, they were not captured in our logs above

<!-- References:
- See proto definition of [TaskStatus](https://github.com/ray-project/ray/blob/releases/2.8.1/src/ray/protobuf/common.proto#L724-L740) -->